# Track data in existing storage locations

You can track data **without** copying, if your data is located in the same location configured as the lamindb storage.

In [ ]:
import lamindb as ln

ln.track()

## Track existing data from local storage

Now let's track a file that is already in the configured local directory: `./mydata`

In [ ]:
configured_storage = ln.setup.settings.instance.storage.root
filepath = ln.dev.datasets.file_mini_csv()
filepath = filepath.rename(configured_storage / filepath.name)
assert configured_storage in filepath.parents

In [ ]:
!ls ./mydata/mini.csv

In [ ]:
file = ln.File("./mydata/mini.csv")

In [ ]:
ln.add(file)

### What happens under the hood?

### In the SQL database

1. A `File` entry
2. A `Notebook` entry
3. A `Run` entry

All three entries are linked so that you can find the file using any of the metadata fields.

In [ ]:
ln.select(ln.File, name=file.name, suffix=file.suffix).one()

In [ ]:
ln.select(ln.schema.Notebook, id=ln.context.transform.id).one()

In [ ]:
ln.select(ln.schema.Run, id=ln.context.run.id).one()

#### In storage

The `./mydata/mini.csv` is untouched, and no additional file is created:

In [ ]:
!ls ./mydata

### Find and retrieve data

In [ ]:
file = ln.select(ln.File, name="mini", suffix=".csv").one()

file

In [ ]:
file.path()

In [ ]:
file.load()

In [ ]:
assert str(filepath.resolve()) == str(file.path())

## Track existing data from cloud storage

### Configure cloud storage

```{tip}

If you already have an existing instance with a different storage, you can switch storage by:

```python
lamin.set.storage({storage_path})
```

Let's configure an instance with cloud storage (s3):

In [ ]:
ln.setup.login("testuser1")
ln.setup.init(storage="s3://lamindb-ci")

Now we'd like to ingest a csv file that is located in this cloud bucket:

In [ ]:
cloudpath = "s3://lamindb-ci/test-data/Species.csv"

In [ ]:
file = ln.File(data=cloudpath)

In [ ]:
id = file.id

In [ ]:
ln.add(file)

### Find and retrieve data

In [ ]:
file = ln.select(ln.File, id=id).one()

file

In [ ]:
df = file.load()

In [ ]:
df.head()

In [ ]:
# [Not for users] Clean the test instance
ln.delete(file, delete_data_from_storage=False)